# Assignment 2

#### First Name: Paul
#### Last Name: Rognon


## 1. Load Data from JSON

In [1]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("Twitter Analysis")\
.getOrCreate()


In [2]:
df_twitter = spark.read.json("corona_tweet_new.json")

In [3]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- contributors_enabled: boolean (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- default_profile: boolean (nullable = true)
 |    |-- default_profile_image: boolean (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- follow_request_sent: string (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- following: string (nullable = true)
 |    |-- frie

In [4]:
from pyspark.sql.functions import col 
first_cols = df_twitter.columns[0:(len(df_twitter.columns)-1)]
df_twitter = df_twitter.select(first_cols+[col("user.id_str").alias("user_id_str"),
                                                col("user.followers_count").alias("user_followers_count"),
                                                col("user.friends_count").alias("user_friends_count"),
                                                col("user.created_at").alias("user_created_at")])
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- user_followers_count: long (nullable = true)
 |-- user_friends_count: long (nullable = true)
 |-- user_created_at: string (nullable = true)



In [5]:
# Print the total count of number of records in df_twitter(1 point)
print(df_twitter.count())

15894


In [6]:
df_twitter.select(col("source")).show(10,False)

+------------------------------------------------------------------------------------+
|source                                                                              |
+------------------------------------------------------------------------------------+
|<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>             |
|<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>|
|<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>|
|<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>|
|<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>|
|<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>             |
|<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                  |
|<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>|
|<a href="http://twitter.com/download/andro

In [7]:
# Extract the source label from source col by droping the anchor tab and save it as another col named extracted_source
# for example <a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a> => Twitter Web App
# you can use "<a [^>]+>([^<]+)" as regualr expresion and the group would be 1 for this regular expression.
#(4 points)

from pyspark.sql.functions import regexp_extract, col
df_twitter = df_twitter.withColumn('extracted_source',
                                    regexp_extract(df_twitter['source'],'<a [^>]+>([^<]+)',1))
df_twitter.select(col('extracted_source'),col('source')).show()

+-------------------+--------------------+
|   extracted_source|              source|
+-------------------+--------------------+
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter Web Client|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
+----------

In [8]:
# Convert the DataFrame into RDD
rdd_twitter=df_twitter.rdd.map(tuple)

In [9]:
# Create a temporay table in memory with name as twitter (1 point)
df_twitter.createOrReplaceTempView("twitter")

## 2. Analyze Data

#### You will be writing code to find the answer to the questions listed below using Just RDD, Using spark SQL 

- Analyze using RDD 
- Analyze using Dataframe without temp table 
- Analyze using spark.sql with temp table


#### 2.1 Get total number of unique users (1 point for each type)

In [10]:
# Using RDD
user_tweet = rdd_twitter.map(lambda s : (s[10], 1))
user_tweet = user_tweet.reduceByKey(lambda a, b : a + b)
user_tweet.count()

14094

In [11]:
# Using DataFrame
df_twitter.select("user_id_str").distinct().count()

14094

In [12]:
# Using spark.sql and the temporay table.
spark.sql("SELECT COUNT (DISTINCT user_id_str) FROM twitter").show()

+---------------------------+
|count(DISTINCT user_id_str)|
+---------------------------+
|                      14094|
+---------------------------+



#### 2.2 Get count of user who have more than 1 tweet in the data (2 points)

In [13]:
# Using RDD
user_tweet.filter(lambda s: s[1] > 1).count()

1016

In [14]:
# Using DataFrame
df_twitter_temp = df_twitter.groupby("user_id_str").count()
df_twitter_temp.filter(df_twitter_temp["count"]>1).count()

1016

In [15]:
# Using spark.sql and the temporay table.
spark.sql("SELECT user_id_str,COUNT(*) AS NB FROM twitter GROUP BY user_id_str").createOrReplaceTempView("twitter_temp")
spark.sql("SELECT COUNT(*) FROM twitter_temp WHERE NB > 1").show()

+--------+
|count(1)|
+--------+
|    1016|
+--------+



#### 2.3 Get total number unique extracted_source (1 point each)

In [16]:
# Using RDD
source_tweet = rdd_twitter.map(lambda s : (s[14], 1))
source_tweet = source_tweet.reduceByKey(lambda a, b : a + b)
source_tweet.count()

133

In [17]:
# Using DataFrame
df_twitter.select("extracted_source").distinct().count()

133

In [18]:
# Using spark.sql and the temporay table.
spark.sql("SELECT DISTINCT extracted_source FROM twitter").count()

133

#### 2.4 Get top 5 most used extracted_source

In [19]:
# Using RDD (5 points)
source_tweet.takeOrdered(5, lambda s: -1 * s[1])

[('Twitter for Android', 6262),
 ('Twitter for iPhone', 5698),
 ('Twitter Web App', 2878),
 ('Twitter for iPad', 428),
 ('Twitter Web Client', 136)]

In [20]:
# Using DataFrame (2 points)
df_twitter.groupBy("extracted_source").count().sort("count", ascending = False).show(5)

+-------------------+-----+
|   extracted_source|count|
+-------------------+-----+
|Twitter for Android| 6262|
| Twitter for iPhone| 5698|
|    Twitter Web App| 2878|
|   Twitter for iPad|  428|
| Twitter Web Client|  136|
+-------------------+-----+
only showing top 5 rows



In [21]:
# Using spark.sql and the temporay table. (2 points)
spark.sql("select extracted_source, COUNT (*) AS NB_2 FROM twitter GROUP BY extracted_source order by NB_2 desc").show(5)

+-------------------+----+
|   extracted_source|NB_2|
+-------------------+----+
|Twitter for Android|6262|
| Twitter for iPhone|5698|
|    Twitter Web App|2878|
|   Twitter for iPad| 428|
| Twitter Web Client| 136|
+-------------------+----+
only showing top 5 rows



#### 2.5 Get count of distinct hastags used ( 5 point each) 

In [22]:
# Using RDD
rdd_twitter.flatMap(lambda s : s[2]).distinct().count()

1215

In [23]:
# Using DataFrame
from pyspark.sql.functions import explode, flatten
df_twitter.select(explode(df_twitter.hashtags)).distinct().count()

1215

In [24]:
# Using spark.sql and the temporay table.
spark.sql("SELECT explode(hashtags) FROM twitter").createOrReplaceTempView("hashtag_temp")
spark.sql("SELECT COUNT(DISTINCT col) FROM hashtag_temp").show()

+-------------------+
|count(DISTINCT col)|
+-------------------+
|               1215|
+-------------------+



#### 2.6 Get top 5 hashtags

In [25]:
# Using RDD (4 points)
hash_dic = rdd_twitter.flatMap(lambda s : s[2]).countByValue()
sorted(hash_dic.items(),reverse=True, key=lambda item: item[1])[0:5]

[('طبق_القدرات_للثانويه_ياريس', 385),
 ('Corona', 319),
 ('OilPrice', 251),
 ('COVID19', 125),
 ('corona', 123)]

In [26]:
# Using DataFrame (2 points)
from pyspark.sql.functions import explode
df_twitter.select(explode(df_twitter.hashtags)).groupBy("col").count().sort(col("count").desc()).show(5)

+--------------------+-----+
|                 col|count|
+--------------------+-----+
|طبق_القدرات_للثان...|  385|
|              Corona|  319|
|            OilPrice|  251|
|             COVID19|  125|
|              corona|  123|
+--------------------+-----+
only showing top 5 rows



In [27]:
# Using spark.sql and the temporay table. (2 points)
spark.sql("""SELECT col, COUNT(col) AS freq 
    FROM hashtag_temp
    GROUP BY col
    ORDER BY freq DESC
    LIMIT 5""").show()

+--------------------+----+
|                 col|freq|
+--------------------+----+
|طبق_القدرات_للثان...| 385|
|              Corona| 319|
|            OilPrice| 251|
|             COVID19| 125|
|              corona| 123|
+--------------------+----+



#### 2.6 Get total number of tweets which are retweeted more than 100 times

In [28]:
# Using RDD
rdd_twitter.filter(lambda s: s[8] > 100).count()

15753

In [29]:
# Using DataFrame
df_twitter.filter(df_twitter["retweet_count"]>100).count()

15753

In [30]:
# Using spark.sql and the temporay table.
spark.sql("select COUNT (*) FROM twitter WHERE retweet_count > 100").show()

+--------+
|count(1)|
+--------+
|   15753|
+--------+



#### 2.8 Get top 3 most retweeted tweets per country (8 points)

In [31]:
# Using RDD
country_retweet = rdd_twitter.map(lambda s : (s[6], (s[3],s[8])))
# RDD with ids of three most retweeted and the number of retweets
country_retweet=country_retweet.groupByKey().mapValues(lambda x: sorted(x,reverse=True, key=lambda x :x[1])[0:3])
country_retweet.take(2)

[('USA',
  [('1252331777806524416', 9994),
   ('1252254239805579264', 9987),
   ('1252335464750735362', 9982)]),
 ('Pakistan',
  [('1252334264248606720', 9988),
   ('1252251912084357121', 9975),
   ('1252252126694309888', 9973)])]

In [32]:
# Using DataFrame
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col,row_number

window = Window.partitionBy(df_twitter['location']).orderBy(df_twitter['retweet_count'].desc())

df_twitter.select("location","id","retweet_count", row_number().over(window).alias('rank')).filter(col('rank') <= 3).show() 

+--------+-------------------+-------------+----+
|location|                 id|retweet_count|rank|
+--------+-------------------+-------------+----+
| Germany|1252334028092399622|         9999|   1|
| Germany|1252330902325248000|         9997|   2|
| Germany|1252252295510855682|         9990|   3|
|   China|1252335780707684352|         9998|   1|
|   China|1252253596516843520|         9993|   2|
|   China|1252255562525560832|         9984|   3|
|   India|1252332114948874240|         9988|   1|
|   India|1252252336921206787|         9976|   2|
|   India|1252254519116746754|         9973|   3|
|   Chile|1252253612140490759|         9988|   1|
|   Chile|1252334891951427585|         9984|   2|
|   Chile|1252253710182481920|         9978|   3|
|   Italy|1252252106750377996|         9994|   1|
|   Italy|1252251206027816960|         9984|   2|
|   Italy|1252330500670337024|         9971|   3|
|   Spain|1252335445876367361|         9992|   1|
|   Spain|1252334839094599681|         9981|   2|


In [33]:
# Using spark.sql and the temporay table.
spark.sql("""SELECT location, id, retweet_count, rank 
FROM (SELECT location, id, retweet_count,ROW_NUMBER() OVER (PARTITION BY location ORDER BY retweet_count DESC) AS rank FROM twitter) tmp 
WHERE rank <= 3 ORDER BY location, rank
""").show()


+--------+-------------------+-------------+----+
|location|                 id|retweet_count|rank|
+--------+-------------------+-------------+----+
|  Canada|1252335430323888128|         9997|   1|
|  Canada|1252254877939531776|         9992|   2|
|  Canada|1252252082825986051|         9987|   3|
|   Chile|1252253612140490759|         9988|   1|
|   Chile|1252334891951427585|         9984|   2|
|   Chile|1252253710182481920|         9978|   3|
|   China|1252335780707684352|         9998|   1|
|   China|1252253596516843520|         9993|   2|
|   China|1252255562525560832|         9984|   3|
| Germany|1252334028092399622|         9999|   1|
| Germany|1252330902325248000|         9997|   2|
| Germany|1252252295510855682|         9990|   3|
|   India|1252332114948874240|         9988|   1|
|   India|1252252336921206787|         9976|   2|
|   India|1252254519116746754|         9973|   3|
|   Italy|1252252106750377996|         9994|   1|
|   Italy|1252251206027816960|         9984|   2|


#### 2.9 Total number of tweets per country

In [34]:
# Using RDD (3 points)
country_tweet = rdd_twitter.map(lambda s : (s[6],1))
country_tweet=country_tweet.reduceByKey(lambda a, b : a + b)
country_tweet.take(10)

[('USA', 1539),
 ('Pakistan', 1470),
 ('Mexico', 1409),
 ('Spain', 1464),
 ('Canada', 1441),
 ('Germany', 1426),
 ('India', 1480),
 ('China', 1457),
 ('UK', 1376),
 ('Chile', 1410)]

In [35]:
# Using DataFrame (2 points)
df_twitter.groupBy("location").count().show()

+--------+-----+
|location|count|
+--------+-----+
| Germany| 1426|
|   India| 1480|
|   China| 1457|
|   Chile| 1410|
|   Italy| 1422|
|   Spain| 1464|
|     USA| 1539|
|  Mexico| 1409|
|      UK| 1376|
|  Canada| 1441|
|Pakistan| 1470|
+--------+-----+



In [36]:
# Using spark.sql and the temporay table. (1 point)
spark.sql("SELECT location, COUNT(*) FROM twitter GROUP BY location ").show()

+--------+--------+
|location|count(1)|
+--------+--------+
| Germany|    1426|
|   India|    1480|
|   China|    1457|
|   Chile|    1410|
|   Italy|    1422|
|   Spain|    1464|
|     USA|    1539|
|  Mexico|    1409|
|      UK|    1376|
|  Canada|    1441|
|Pakistan|    1470|
+--------+--------+



## 3. Save Data

#### 3.1 save the data such that you have seperate folder per country (2 points)

In [37]:
# Using DataFrame
df_twitter.write.partitionBy("location").format("json").save("tweetsByLocation")

#### 3.2 Save the data as parquet files (1 points)

In [38]:
# Using DataFrame
df_twitter.write.save("tweets.parquet", format="parquet")